# SciToken Demo
In this demo, you will learn how to: 
- Request a demo credential from the SciTokens demo issuer.
- Examine a token
- Validate a token
- A bonus exercise where you can earn a **Badge!**

This demo is to give an idea what it is like to develop with SciTokens.

In [ ]:
# You don't need to install the scitokens library in Binder, it's already installed for you.
# !pip install scitokens
import json, requests, scitokens

## Getting your first token
We will first create a simple function to retrieve a token from [demo.scitokens.org](https://demo.scitokens.org).  **Normally** it would require a private key, as well as a public key posted on a website.  For now, we will use [demo.scitokens.org](https://demo.scitokens.org) since it is easier.  Most issuers will use oauth to get a token.

In [ ]:
# Specify an algorithm for signature
# ES256 = Elliptic Curve with SHA-256
# getToken will return a signed token with the payload
def getToken(payload: dict):
    data = json.dumps({'algorithm': "ES256", 'payload': payload})
    resp = requests.post("https://demo.scitokens.org/issue", data=data)
    return resp.text

Lets create a simple token with your email address.  Enter your email address below to create a personalized token.  This will print out the token in it's raw (base64 encoded) form.

In [ ]:
payload = {
    "sub": "<email address>"
}
print(getToken(payload))

## Parsing the token
When we deserialize the token, we have to give the audience.  When deserializtion happens, it will also validate some of the attributes, such as the token expiration and the audience.

In [ ]:
raw_token = getToken(payload)
parsed_token = scitokens.SciToken.deserialize(raw_token, audience="https://demo.scitokens.org")

In [ ]:
# Print out all of the claims in the token
for claim in parsed_token.claims():
    print(claim)

What happens if you edit the token?  It should fail the signature check.  There will be a very, very large error.

In [ ]:
raw_token += "blah"
parsed_token = scitokens.SciToken.deserialize(raw_token, audience="https://demo.scitokens.org")

In [ ]:
# In the python library, you can edit claims like a dictionary.
# But this doesn't have any effect, since you don't have the private key to sign it
parsed_token['scope'] = "read:/protected"
parsed_token.serialize()

## Getting an updated token
Since we updated the payload, the signature of the token no longer matches.  In fact, try running the `token.serialize` function.  You need a private key to serialize a token since it must be signed.

We can, however, request a new token from demo.scitokens.org with the `scope` = `read:/protected`

In [ ]:
payload = {
    "sub": "<email address>",
    "scope": "read:/protected"
}
token = getToken(payload)

## Verifying the token against an authorization
Act as if you are a storage server.  When a request comes in with a scitoken, the request will want to read or write to some file.  The token permissions are encoded in the `scope` attribute.  First, lets take the token we created in the last call and try some authorization tests.

In [ ]:
token = getToken(payload)
parsed_token = scitokens.SciToken.deserialize(token, audience="https://demo.scitokens.org")

In [ ]:
# Create an enforcer which is a high level way to "query" the token
# The C++ library has a very similar method
enforcer = scitokens.Enforcer(issuer="https://demo.scitokens.org", audience="https://demo.scitokens.org")

In [ ]:
enforcer.test(parsed_token, "read", "/protected")

In [ ]:
enforcer.test(parsed_token, "read", "/doesnotexist")

In [ ]:
enforcer.test(parsed_token, "read", "/protected/file.txt")

In [ ]:
enforcer.test(parsed_token, "read", "/protected/dir1/dir2/file.dat")

In [ ]:
enforcer.test(parsed_token, "write", "/protected/dir1/dir2/file.dat")

## How to query a web resource
The token is passed over HTTP in the Authorization header:

Header:

    ...
    Host: demo.scitokens.org
    Authorization: Bearer <token>
    ...

 

In [ ]:
headers = {'Authorization': "Bearer " + token}
query = requests.get("https://demo.scitokens.org/protected", headers=headers)
print(query.text)

In [ ]:
query = requests.get("https://demo.scitokens.org/secret", headers=headers)
print(query.text)

To query a resource with Curl, the command would be:

    $ curl -H "Authorization: Bearer <token>" https://demo.scitokens.org/protected

In [ ]:
print("curl -H \"Authorization: Bearer " + token + "\" https://demo.scitokens.org/protected".format(token))

## Bonus Exercise!
In this bonus exercise, you will request a token that is able to access https://demo.scitokens.org/secret.  Things to think about:
- What does the `scope` claim need to be?
- Include your email address in the `sub` attribute for a [OpenBadge badge](https://badgr.com/public/badges/0xFqlz4bQ5qAd7FG6FIwEQ)!